In [114]:
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
from itertools import chain

my_url = 'https://www.projekt-gutenberg.org/fallada/hoppelpo/chap001.html'

uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()

page_soup = soup(page_html, 'html.parser')

In [115]:
container = page_soup.find_all('p')
container = container[0]

In [116]:
title = container.find_all('h3')
title = str(title[0])

In [117]:
story = container.find_all('p')
paragraphs = []
for i in story:
    paragraphs.append(str(i))

In [118]:
for i in range(len(paragraphs)):
    paragraphs[i] = paragraphs[i].replace('<p>', '')
    paragraphs[i] = paragraphs[i].replace('</p>', '') 

In [121]:
words = []
try:
    for i in range(len(paragraphs)):
        for j in range(len(paragraphs[i])):
            words.append(paragraphs[j].split(' ')) 
except:
    pass

In [125]:
words = list(chain.from_iterable(words))

In [138]:
for i in range(len(words)):
        words[i] = words[i].strip()
        words[i] = words[i].replace('\xa0...', '')
        words[i] = words[i].replace('<a>', '')
        words[i] = words[i].replace('</a>', '')
        title="Dieter7/lac"/a'
        id="page9"
        name="page9"
        title="suru/Dieter7"/a'
        'title="lac/Dieter7"/a'
        words[i] = words[i].replace('-', '')
        words[i] = words[i].replace('–', '')
        words[i] = words[i].replace('.', '')
        words[i] = words[i].replace(':', '')
        words[i] = words[i].replace(',', '')
        words[i] = words[i].replace('>', '')
        words[i] = words[i].replace('<', '')
        words[i] = words[i].replace('›', '')
        words[i] = words[i].replace('‹', '')
        words[i] = words[i].replace('?', '')
        words[i] = words[i].replace('!', '')
        words[i] = words[i].replace('«', '')
        words[i] = words[i].replace('»', '')
        words[i] = words[i].replace(' ', '')

In [134]:
# for i in range(len(words)):
#     for j in range(len(words[i])):
#         words[i][j] = words[i][j].strip()
#         words[i][j] = words[i][j].replace('\xa0...', '')
#         words[i][j] = words[i][j].replace('-', '')
#         words[i][j] = words[i][j].replace('–', '')
#         words[i][j] = words[i][j].replace('.', '')
#         words[i][j] = words[i][j].replace(':', '')
#         words[i][j] = words[i][j].replace(',', '')
#         words[i][j] = words[i][j].replace('>', '')
#         words[i][j] = words[i][j].replace('<', '')
#         words[i][j] = words[i][j].replace('?', '')
#         words[i][j] = words[i][j].replace('!', '')
#         words[i][j] = words[i][j].replace('«', '')
#         words[i][j] = words[i][j].replace('»', '')
#         words[i][j] = words[i][j].replace(' ', '')

In [145]:
words[500:-500]

['Zug',
 'fährt',
 'weiter',
 'nach',
 'Polen',
 'und',
 'die',
 'werden',
 'uns',
 'gerade',
 'einen',
 'Hoppelpoppel',
 'zurückschicken',
 'Am',
 'nächsten',
 'Morgen',
 'machte',
 'der',
 'Vater',
 'seine',
 'Verlustanzeige',
 'dann',
 'kam',
 'der',
 'Nachmittagsschlaf',
 '',
 'aber',
 'nein',
 'es',
 'kam',
 'kein',
 'Nachmittagsschlaf',
 'Hoppelpoppel',
 'Hoppelpoppel',
 'kommt',
 'bald',
 'Nun',
 'Gleich',
 'Thomas',
 'muß',
 'schlafen',
 'Gebrüll',
 'Wut',
 'Trostlosigkeit',
 'Jammer',
 'nur',
 'kein',
 'Schlaf',
 'Und',
 'am',
 'Abend',
 'dasselbe',
 'Das',
 'neue',
 'Häuserchen',
 'und',
 'das',
 'viele',
 'Wasser',
 'und',
 'der',
 'Garten',
 'und',
 'der',
 'Hund',
 'im',
 'Zwinger',
 'und',
 'die',
 'vielen',
 'Dampfer',
 '',
 'alles',
 'nichts',
 'Hoppelpoppel',
 'lieber',
 'Hoppelpoppel',
 '',
 'wo',
 'bist',
 'du',
 'Hoppelpoppel',
 'ein',
 'alberner',
 'schwarzer',
 'Stoffhund',
 'war',
 'eine',
 'finstere',
 'Wolke',
 'am',
 'Himmel',
 'nach',
 'drei',
 'Tagen',
 'übe

In [95]:
story

[<p>Es war einmal ein kleiner Junge, der hieß Thomas. Dem hatten seine Großeltern zum ersten Weihnachtsfest einen kleinen Hund aus schwarzem Plüsch geschenkt, mit Hängeohren und frechen braunen Augen, eine Art Dackeltier, aber auf Rädern. Und da die Achsen dieser Räder nicht im Mittelpunkt saßen, sondern seitlich, hoppelte und wogte das schwarze Stoffgeschöpf auf und nieder, als haste es wild und über alle Kraft imaginären Hasen nach. Darum taufte der Vater den Hund »Hoppelpoppel«, und als Thomas etwas älter geworden war und sprechen konnte, genehmigte auch er diesen Namen. Er liebte den Hund sehr, immer mußte er bei ihm sein, auch im Schlaf durfte er ihn nicht verlassen, und er wachte sehr genau darüber, daß die Eltern nicht nur ihrem Sohn, sondern auch dem Hoppelpoppel gute Nacht sagten. Es war eben eine richtige Liebe.</p>,
 <p>Nun geschah es, daß Toms Eltern an einen neuen Wohnsitz verzogen, weit, weit weg. Der kleine Thomas blieb während der Umzugstage bei der guten Tante »Kunjä«,

In [26]:
st = []
i = 1
for i in len(paragraphs):
    st.append(str(i))

TypeError: 'int' object is not iterable

In [20]:
paragraphs[53]

'<p>Komisch: plötzlich war die Welt wieder in Ordnung. \n   <a id="page9" name="page9" title="Dieter7/lac"></a></p>'

In [ ]:
p = soup.find_all('p')
paragraphs = []
for x in p:
    paragraphs.append(str(x))